In [29]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from tqdm import tqdm
from combine_svm import Phase_difference, Remove_impulse_noise, Amplify_signal, iir_bandpass_filter_1, MLR, feature_detection, feature_compress, candidate_search

In [60]:
def bmi(sig):
    """
    Return bmi array of given raw_sig.

    Parameters
    ----------
    start_min & start_sec:
        紀錄測量到的第一個數據
    coco:
        判斷是否已經完成忽略起始未滿 60 秒的數據
    number_i:
        計算 Sampling rate 的 Index
    bi_arr:
        最後輸出的 BMI 陣列
    amp_sec:
        計算每秒輸出的平均，因為取樣頻率有差異所以這樣做
    window_sig:
        存放長度為 60 秒的資料
    window_sig_piece:
        以大小為 10s 的窗格處理 window_sig 因此長度為 6

    Returns
    -------
    out : ndarray
        bmi array of given raw_sig.
    """
    start_min = sig.datetime[0][3:5]
    start_sec = sig.datetime[0][6:8]
    coco = False
    number_i = 0
    bi_arr = []
    window_sig = []
    window_sig_piece = []
    sampling_rate_arr = np.ones(60)
    for index in tqdm(range(len(sig))):

        # 為了保持 60 秒的時間區間，一開始當秒數不等於 00 時皆刪去
        if coco == False and start_sec != "00":
            start_min = sig.datetime[index][3:5]
            start_sec = sig.datetime[index][6:8]
            bi_arr.append(0)
        else:
            coco = True

        if coco:
            if start_min == sig.datetime[index][3:5]:
                window_sig.append(sig.unwrapPhasePeak_mm[index])
                if start_sec == sig.datetime[index][6:8]:
                    sampling_rate_arr[number_i] += 1
                    bi_arr.append(0)
                else:
                    number_i += 1
                    start_sec = sig.datetime[index][6:8]
                    bi_arr.append(0)

            else:
                start_min = sig.datetime[index][3:5]
                start_sec = sig.datetime[index][6:8]
                amp_sec = []
                num_start = 0
                for numbers in sampling_rate_arr:
                    num_end = int(numbers) + num_start
                    amp_sec.append(np.mean(window_sig[num_start:num_end]))  # Paper 沒寫暫時用 mean
                    num_start = num_end

                window_size = 10
                for i in range(6):
                    ak = np.mean(amp_sec[i*window_size:(i+1)*window_size])  # Paper 沒寫暫時用 mean
                    window_sig_piece.append(ak)

                    if i == 0:
                        a_min = ak
                    if a_min > ak:
                        a_min = ak

                # 計算BI(k)
                bi = 0
                for i in range(6):
                    tmp = window_sig_piece[i]
                    bi += (tmp - a_min)

                # 得到的特徵
                bi_arr.append(bi)

                # Reset
                number_i = 0
                window_sig = []
                window_sig_piece = []
                sampling_rate_arr = np.ones(60)
    return bi_arr

# 需先將單位改成秒
def var_RPM(sig):
    breath = sig["breath"]
    N = 10 * 60  # 預設為 10min = 10 * 60s
    var_RPM_k = []

    # 過程補 0
    for i in range(N):
        var_RPM_k.append(0)

    # 窗格大小: 10 (窗格重疊)
    for k in tqdm(range(len(breath) - N)):
        window_sum = 0
        tmp = np.copy(breath[k:k+N])
        tmp_mean = np.mean(tmp)
        for j in range(N):
            tmp_a = tmp[j] - tmp_mean
            if tmp_a == 0:
                window_sum += 0
            else:
                window_sum += np.square(tmp_a)
        var_RPM_k.append(window_sum / (N - 1))

    # 窗格大小: 10 (窗格不重疊)
    # for k in range(len(sig) // N):
    #     window = 0
    #     tmp = breath[k*10:(k+1)*10]
    #     tmp_mean = np.mean(tmp)
    #     for j in range(N):
    #         window += np.square(tmp[j] - tmp_mean)
    #     var_RPM_k.append(window / (N - 1))
    
    return var_RPM_k

def ada_assist(sig, brhr):

    # 0: 呼吸, 1: 心跳
    if brhr == 0:
        a = [1.5, 0.125, 0.55, 20, 5, 2, 22, 17]
    else:
        a = [1.5, 0.9, 1.9, 20, 9, 2, 5, 4]

    # Phase difference
    phase_diff = Phase_difference(sig)

    # RemoveImpulseNoise
    re_phase_diff = Remove_impulse_noise(phase_diff, int(a[0]))

    # Linear amplify
    amp_sig = Amplify_signal(re_phase_diff)

    # Bandpass signal (cheby2)
    bandpass_sig = iir_bandpass_filter_1(amp_sig, float(a[1]), float(a[2]), int(a[3]), int(a[4]), "cheby2") # Breath: 0.1 ~ 0.33 order=5, Hreat: 0.8 ~ 2.3
 
    # Smoothing signal
    smoothing_signal = MLR(bandpass_sig, int(a[5]))  # Breath = 9, Heart = 6, Delta = 1

    #detect the feature
    feature_peak, feature_valley, feature_sig = feature_detection(smoothing_signal) #找出所有的波峰及波谷

    #compress with window size 7
    compress_peak, compress_valley = feature_compress(feature_peak, feature_valley, int(a[6]), smoothing_signal)  # Br: 20 Hr: 6  ex: 25

    # Feature sort
    compress_feature = np.append(compress_peak, compress_valley)
    compress_feature = np.sort(compress_feature)

    # Candidate_search
    NT_points, _ = candidate_search(smoothing_signal, compress_feature, int(a[7]))  # breath = 18 hreat = 4 ex: 7

    return NT_points

def ada(sig, brhr):
    """
    每分鐘更新，ADA
    """
    unwrapPhase = sig['unwrapPhasePeak_mm'].values
    start_min = sig.datetime[0][3:5]
    start_sec = sig.datetime[0][6:8]
    coco = False
    ada_arr = []
    window_sig = []
    number_i = 0
    sampling_rate_arr = np.ones(60)
    for index in tqdm(range(len(unwrapPhase))):

        # 為了保持 60 秒的時間區間，一開始當秒數不等於 00 時皆刪去
        if coco == False and start_sec != "00":
            start_min = sig.datetime[index][3:5]
            start_sec = sig.datetime[index][6:8]
            ada_arr.append(0)
        else:
            coco = True

        # ----------------------------------------------------
        if coco:
            if start_min == sig.datetime[index][3:5]:
                window_sig.append(sig.unwrapPhasePeak_mm[index])

                if start_sec == sig.datetime[index][6:8]:
                    sampling_rate_arr[number_i] += 1
                    ada_arr.append(0)
                    
                else:
                    number_i += 1
                    start_sec = sig.datetime[index][6:8]
                    ada_arr.append(0)
            else:
                diff = 0
                start_min = sig.datetime[index][3:5]
                start_sec = sig.datetime[index][6:8]
                top = ada_assist(window_sig, brhr)  # 0 for breath, 1 for heart.
                for top_index in range(len(top) - 1):
                    cur_index = int(top[top_index + 1])
                    pre_index = int(top[top_index])
                    diff += math.fabs(window_sig[cur_index] - window_sig[pre_index])
                ada_arr.append(diff)
                number_i = 0
                window_sig = []
                sampling_rate_arr = np.ones(60)
    return ada_arr

# 需先將單位改成秒
def rem_parameter(sig):
    q = 2
    start_min = sig.datetime[0][3:5]
    start_sec = sig.datetime[0][6:8]
    coco = False
    resp = sig["breath"].values
    for index in tqdm(range(q, len(resp) - q)):

        # 為了保持 60 秒的時間區間，一開始當秒數不等於 00 時皆刪去
        if coco == False and start_sec != "00":
            start_min = sig.datetime[index-q][3:5]
            start_sec = sig.datetime[index-q][6:8]
            ada_arr.append(0)
        else:
            coco = True

        # ----------------------------------------------------
        if coco:
            tmp = np.copy(resp[index-q:index+q+1])
            print(tmp)
    return 132


In [7]:
raw_data_pd = pd.DataFrame()
raw_sig = pd.read_csv("./0.csv")
raw_sig['heart'] = raw_sig['heart'].replace(0, np.nan)
new_sig = raw_sig.dropna().reset_index()
var_RPM_k = var_RPM(new_sig)
new_sig["var_RPM"] = var_RPM_k
new_sig.to_csv("123321.csv", index=False)

100%|██████████| 28045/28045 [00:19<00:00, 1420.28it/s]


In [52]:
# Amplitude Difference Accumulation (ADA) of Respiration:
raw_sig = pd.read_csv("./0.csv")
ada_arr = ada(raw_sig, 0)  # brhr: 0(呼吸) 1(心跳)
raw_sig["ada_br"] = ada_arr
raw_sig.to_csv("123321.csv", index=False)


100%|██████████| 572898/572898 [00:15<00:00, 36508.86it/s]


In [61]:
# REM Parameter
raw_sig = pd.read_csv("./0.csv")
raw_sig['heart'] = raw_sig['heart'].replace(0, np.nan)
new_sig = raw_sig.dropna().reset_index()
rem = rem_parameter(new_sig)
# new_sig["rem_parameter"] = rem
# new_sig.to_csv("123321.csv", index=False)

  3%|▎         | 939/28641 [00:00<00:02, 9316.51it/s]

[15.8657 15.75   15.6987 15.5845 15.5259]
[15.75   15.6987 15.5845 15.5259 15.4233]
[15.6987 15.5845 15.5259 15.4233 15.3267]
[15.5845 15.5259 15.4233 15.3267 15.2388]
[15.5259 15.4233 15.3267 15.2388 15.3193]
[15.4233 15.3267 15.2388 15.3193 15.457 ]
[15.3267 15.2388 15.3193 15.457  15.5874]
[15.2388 15.3193 15.457  15.5874 15.7104]
[15.3193 15.457  15.5874 15.7104 15.7969]
[15.457  15.5874 15.7104 15.7969 15.8613]
[15.5874 15.7104 15.7969 15.8613 16.0078]
[15.7104 15.7969 15.8613 16.0078 16.0635]
[15.7969 15.8613 16.0078 16.0635 16.1353]
[15.8613 16.0078 16.0635 16.1353 16.2056]
[16.0078 16.0635 16.1353 16.2056 16.2935]
[16.0635 16.1353 16.2056 16.2935 16.3623]
[16.1353 16.2056 16.2935 16.3623 16.3975]
[16.2056 16.2935 16.3623 16.3975 16.377 ]
[16.2935 16.3623 16.3975 16.377  16.3491]
[16.3623 16.3975 16.377  16.3491 16.2949]
[16.3975 16.377  16.3491 16.2949 16.2363]
[16.377  16.3491 16.2949 16.2363 16.1777]
[16.3491 16.2949 16.2363 16.1777 16.1191]
[16.2949 16.2363 16.1777 16.1191 1

 10%|▉         | 2759/28641 [00:00<00:03, 8615.53it/s]


[14.8096 14.8096 14.8096 14.8096 14.7949]
[14.8096 14.8096 14.8096 14.7949 14.751 ]
[14.8096 14.8096 14.7949 14.751  14.7231]
[14.8096 14.7949 14.751  14.7231 14.6968]
[14.7949 14.751  14.7231 14.6968 14.6821]
[14.751  14.7231 14.6968 14.6821 14.6411]
[14.7231 14.6968 14.6821 14.6411 14.376 ]
[14.6968 14.6821 14.6411 14.376  14.2646]
[14.6821 14.6411 14.376  14.2646 14.4009]
[14.6411 14.376  14.2646 14.4009 14.4727]
[14.376  14.2646 14.4009 14.4727 14.5854]
[14.2646 14.4009 14.4727 14.5854 14.439 ]
[14.4009 14.4727 14.5854 14.439  14.4097]
[14.4727 14.5854 14.439  14.4097 14.4507]
[14.5854 14.439  14.4097 14.4507 14.5107]
[14.439  14.4097 14.4507 14.5107 14.4609]
[14.4097 14.4507 14.5107 14.4609 14.4375]
[14.4507 14.5107 14.4609 14.4375 14.4668]
[14.5107 14.4609 14.4375 14.4668 14.4961]
[14.4609 14.4375 14.4668 14.4961 14.5151]
[14.4375 14.4668 14.4961 14.5151 14.5444]
[14.4668 14.4961 14.5151 14.5444 14.5459]
[14.4961 14.5151 14.5444 14.5459 14.2397]
[14.5151 14.5444 14.5459 14.2397 

 16%|█▌        | 4464/28641 [00:00<00:02, 8360.82it/s]

...

 54%|█████▍    | 15469/28641 [00:01<00:01, 8342.64it/s]


[18.5229 18.46   18.3208 18.3062 18.3062]
[18.46   18.3208 18.3062 18.3062 18.1245]
[18.3208 18.3062 18.3062 18.1245 18.1963]
[18.3062 18.3062 18.1245 18.1963 18.1392]
[18.3062 18.1245 18.1963 18.1392 18.    ]
[18.1245 18.1963 18.1392 18.     17.8389]
[18.1963 18.1392 18.     17.8389 17.6631]
[18.1392 18.     17.8389 17.6631 17.4653]
[18.     17.8389 17.6631 17.4653 17.3232]
[17.8389 17.6631 17.4653 17.3232 17.3877]
[17.6631 17.4653 17.3232 17.3877 17.5181]
[17.4653 17.3232 17.3877 17.5181 17.6353]
[17.3232 17.3877 17.5181 17.6353 17.6206]
[17.3877 17.5181 17.6353 17.6206 17.5137]
[17.5181 17.6353 17.6206 17.5137 17.2148]
[17.6353 17.6206 17.5137 17.2148 17.0728]
[17.6206 17.5137 17.2148 17.0728 17.0479]
[17.5137 17.2148 17.0728 17.0479 16.9028]
[17.2148 17.0728 17.0479 16.9028 16.8735]
[17.0728 17.0479 16.9028 16.8735 16.8442]
[17.0479 16.9028 16.8735 16.8442 16.8193]
[16.9028 16.8735 16.8442 16.8193 16.8193]
[16.8735 16.8442 16.8193 16.8193 16.793 ]
[16.8442 16.8193 16.8193 16.793  

 58%|█████▊    | 16597/28641 [00:01<00:01, 8387.55it/s]


[17.3862 17.1636 16.9424 16.7271 16.5601]
[17.1636 16.9424 16.7271 16.5601 16.3447]
[16.9424 16.7271 16.5601 16.3447 16.1763]
[16.7271 16.5601 16.3447 16.1763 15.9551]
[16.5601 16.3447 16.1763 15.9551 15.8555]
[16.3447 16.1763 15.9551 15.8555 15.8188]
[16.1763 15.9551 15.8555 15.8188 15.6592]
[15.9551 15.8555 15.8188 15.6592 15.7148]
[15.8555 15.8188 15.6592 15.7148 15.6914]
[15.8188 15.6592 15.7148 15.6914 15.7881]
[15.6592 15.7148 15.6914 15.7881 15.5244]
[15.7148 15.6914 15.7881 15.5244 15.4482]
[15.6914 15.7881 15.5244 15.4482 15.3604]
[15.7881 15.5244 15.4482 15.3604 15.3882]
[15.5244 15.4482 15.3604 15.3882 15.4277]
[15.4482 15.3604 15.3882 15.4277 15.5625]
[15.3604 15.3882 15.4277 15.5625 15.6211]
[15.3882 15.4277 15.5625 15.6211 15.6372]
[15.4277 15.5625 15.6211 15.6372 15.8335]
[15.5625 15.6211 15.6372 15.8335 15.8042]
[15.6211 15.6372 15.8335 15.8042 15.8481]
[15.6372 15.8335 15.8042 15.8481 15.4966]
[15.8335 15.8042 15.8481 15.4966 15.145 ]
[15.8042 15.8481 15.4966 15.145  

KeyboardInterrupt: 